In [ ]:
##################################
#The notebook needed
%matplotlib inline 
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

#!/usr/bin/env python
# coding: utf-8
import os
import sys
sys.path.append("./OASIS-master/")
from oasis.functions import deconvolve

import numpy as np
import matplotlib.pyplot as plt
import sklearn
import h5py
import pdb
from importlib import reload  # Python 3.4+ only.
        
import pickle
import numpy, scipy.io

import size_contrast_analysis as sca
import scipy.stats as sst
from sim_utils import *
import pyute as ut

outputDir='./Output'
if not os.path.exists(outputDir):
    os.makedirs(outputDir)
    
    
# paths
dsbase = '/Users/dan/Documents/notebooks/mossing-PC/shared_data/'

# dsnames = [dsbase+x+'_data_struct.hdf5' for x in ['pyr_l23','sst_l23','vip_l23']]


In [ ]:
#######################
#parameters

nsize,ncontrast = 5,6
to_exclude = ['session_'+exptname for exptname in ['180714_M9053','180321_M7955', '180519_M8959', '180531_M8961',\
                                                 '180618_M8956','190202_M10075', '190620_M10619','200103_M0388']]

#######################
#Files to analyze

for ctr_cutoff in [11]: #

    for pval_cutoff in [0.05]: #

        for running_pct_cutoff in [0.1]: #


            for lt in [False,True]:

                dsnames = [dsbase+x+'_data_struct.hdf5' for x in ['pyr_l4','pyr_l23','pv_l23','sst_l23','vip_l23']]



                ncelltypes = len(dsnames)
                selection = [None for itype in range(ncelltypes)]
                for itype in range(ncelltypes):
                    with ut.hdf5read(dsnames[itype]) as ds:
                        keylist = list(ds.keys())
                        nexpt = len(keylist)
                        to_keep = np.array([k not in to_exclude for k in keylist])
                        selection[itype] = np.arange(nexpt)[to_keep]



                #########################################################
                #########################################################
                # Run

                rs =gen_rs(dsnames=dsnames,selection=selection,dcutoff=ctr_cutoff,pval_cutoff=pval_cutoff,slices=None,running=lt,fill_nans_under_cutoff=True,running_pct_cutoff=running_pct_cutoff)
                #r_spatial=gen_rspatial(dsnames=dsnames,selection=selection,dcutoffs=[0,5,10,15],pval_cutoff=pval_cutoff,slices=None,datafield='decon')
                #########################################################
                paramstring='-running_'+str(lt)+'-pval_cutoff_'+str(pval_cutoff)+'-ctrcutoff_'+str(ctr_cutoff)+'-running_pct_cutoff_'+str(running_pct_cutoff)


                #Save rs

                with open(outputDir+'rs'+paramstring+'.pickle', 'wb') as handle:
                    pickle.dump(rs, handle, protocol=pickle.HIGHEST_PROTOCOL)

                scipy.io.savemat(outputDir+'rs'+paramstring+'.mat', mdict={'rs': rs})    




# Locomotion modulation

In [ ]:
paramstring_loco='-running_'+str(True)+'-pval_cutoff_'+str(0.05)+'-ctrcutoff_'+str(11)    +'-running_pct_cutoff_'+str(0.4)
with open(outputDir+'rs'+paramstring_loco+'.pickle', 'rb') as handle_loco:
    rs_locomotion=pickle.load(handle_loco)


paramstring_stat='-running_'+str(False)+'-pval_cutoff_'+str(0.05)+'-ctrcutoff_'+str(11)     +'-running_pct_cutoff_'+str(0.4)
with open(outputDir+'rs'+paramstring_stat+'.pickle', 'rb') as handle_stat:
    rs_stationay=pickle.load(handle_stat)
    
    

this_pops={'E','P','S','V'};
modal_ucontrast=[0, 6, 12, 25 ,50, 100];

c_l=len(modal_ucontrast);
n_l=4;
s_l=0
fig, axs = plt.subplots(n_l,len(modal_ucontrast), figsize=(12, 12),dpi= 100, facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = .5, wspace=.3)


for k in range(n_l):
    for i in range(c_l):


        runing=rs_locomotion[k+1][0][:,s_l,i]
        stationary=rs_stationay[k+1][0][:,s_l,i]

        diffs=runing-stationary;
        diffs=diffs[~np.isnan(diffs)]


        bins=np.linspace(np.percentile(diffs,5),np.percentile(diffs,95),15);
        axs[k,i].hist(diffs,bins)
        axs[k,i].set_xlim([-0.1, 0.1])

        axs[k,i].axvline(0,c='k',linestyle='dashed')
        axs[k,i].axvline(np.nanmean(diffs),c='k')



In [ ]:
rs_locomotion[0][0].shape
